In [1]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, VBox, HBox
import copy

global selectedSlot
global selectedNum

selectedSlot = None
selectedNum = ''

global buttons
buttons = {}

def isValidMove(grid, row, column, number):
    for x in range(9):
        if grid[row][x] == number:
            return False
        
    for x in range(9):
        if grid[x][column] == number:
            return False
        
    cornerRow = row - row % 3
    cornerCol = column - column % 3
    for x in range(3):
        for y in range(3):
            if grid[cornerRow + x][cornerCol + y] == number:
                return False
            
    return True

def solve(grid, row, column):
    if column == 9:
        if row == 8:
            return True
        row += 1
        column = 0
    
    if grid[row][column] > 0:
        return solve(grid, row, column + 1)
    
    for num in range(1, 10):
        if isValidMove(grid, row, column, num):
            grid[row][column] = num
            
            if solve(grid, row, column + 1):
                return True
        
        grid[row][column] = 0
    
    return False

def fillBoard(grid, board):
    row = 0
    column = 0
    start = 0
    for box in board.children:
        row = (start//3)*3
        column = (start%3)*3
        
        for square in box.children:
            square.on_click(selectSlot)
            buttons[square] = (row, column)
            if grid[row][column] != 0:
                square.description = str(grid[row][column])
                square.disabled=True
            else:
                square.description = str(grid[row][column])
            column = column + 1
            
            if column > (start%3)*3 + 2:
                column = (start%3)*3
                row = row + 1
            
        start = start + 1
        columnn = (start%3) * 3
            
def selectNum(item):
    global selectedNum
    selectedNum = str(item.description)
    enterBoard(selectedNum)
    
def selectSlot(item):
    global selectedSlot 
    if selectedSlot:
        if selectedSlot.style.button_color != 'lightgreen' and selectedSlot.style.button_color != 'red':
            selectedSlot.style.button_color = 'white'
    item.style.button_color = 'lightgray'
    selectedSlot = item
    
def enterBoard(num):
    global buttons
    if None != selectedSlot:
        selectedSlot.description = num

        col = int(buttons[selectedSlot][1])
        row = int(buttons[selectedSlot][0])

        if str(num) == str(solvedGrid[col][row]):
            selectedSlot.style.button_color = 'lightgreen'
        else:
            selectedSlot.style.button_color = 'red'
    
grid = [[7, 0, 0, 0, 2, 0, 4, 8, 0],
        [2, 0, 6, 0, 0, 8, 0, 0, 5],
        [5, 0, 0, 9, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 5, 0, 0, 0, 0],
        [0, 2, 0, 0, 0, 0, 0, 6, 0],
        [0, 0, 0, 0, 6, 7, 0, 0, 0],
        [0, 0, 0, 0, 0, 6, 0, 0, 3],
        [6, 0, 0, 5, 0, 0, 1, 0, 4],
        [0, 9, 3, 0, 4, 0, 0, 0, 7]]

solvedGrid = copy.deepcopy(grid)
num_row = [Button(description=str(i), layout=Layout(width='62.5px', height='62.5px', border='1px solid black')) for i in range(1, 10, 1)]

for button in num_row:
    button.on_click(selectNum)
    
if not solve(solvedGrid, 0, 0):
    print("No Solution Found")
    board = GridBox()
else:
    board = GridBox(children=[GridBox(children=[Button(description='0',layout=Layout(width='97%',height = '97%', border='1px solid black'),
                             style=ButtonStyle(button_color='white')) for i in range(9)
                                           ],
                                  
                            layout=Layout(width='100%', height = '100%', border='2px solid black', 
                                         grid_template_columns='auto auto auto',
                                         grid_template_rows='auto auto auto',
                                         grid_gap='2px 2px'),
                            style=ButtonStyle(button_color='white')) for i in range(9)
                                               ],
            layout=Layout(
                grid_template_columns='200px 200px 200px',
                grid_template_rows='200px 200px 200px',
                grid_gap='0px 0px')
           )
        
    fillBoard(grid, board)
    
top_board = VBox([board])
bottom_nums = HBox(num_row)
VBox([top_board, bottom_nums])